In [ ]:
import sys
sys.path.append('../applications/')
sys.path.append('../measutils/')
sys.path.append('../warpspeed/')


import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
import matplotlib.pyplot as plt
import measure_metric.measureMetric as measureMetric

from stencilgen.stencil import *
from tsmgen.kernel import *
import stencilgen.bench as stencilbench
import tsmgen.benchmark as tsmbench
from predict import *

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport stencilgen.stencil
%aimport stencilgen.bench
%aimport predict

# Goal


                                         launch config
                                           |
                                           v             
    Generator -> Perf Characteristics -> Prediction  
        |
        v
      Code
  
  
- Predict performance from a few code characteristics that any code generator can generate alongside the code
- No look at the actual code, no attempt to derive data from code
- Limitation to kernels:
    - no (complex) control flow
    - no indirect accesses
- Perf characteristics:
    - load/stores with expression for the address dependent only on thread ID
    - number of FP operations
    - (dependence of FP ops)
    - (# of Int operations)
- Launch config: 
    - block size
    - grid size

## Exmaples for Generators: stencils and TSMs

In [ ]:
kernel =  Kernel2DBoxStencil(stencil_range = 1, l1only=True, singleTid=True)

In [ ]:
print( kernel.text)

In [ ]:
stencilbench.printSASS(kernel.text)

In [ ]:
tsmkernel = Kernel(64, 64, 2, 2, 128, dtype="double", transposed=True, leapFrog=False)
print(tsmkernel.text)

In [ ]:
tsmbench.printSASS(tsmkernel.text)

## Phase Model

 - Compiled kernels usually have a typical structure with distinct phases:
  
       INT
       LD 
       FP 
       ST (optional)
       jb/exit 
   
 - INT ususally contains address calculations
 - All LD instructions are moved to the front of the loop/program for maximum overlap of memory latencies
 - Phases may overlap by some amount for larger kernels, to save on registers  
 

## Execution phases

    -                      -
    | (Tint)               |
    -      –               |
    |      |               |
    | Tld  |               |
    |      |               |
    –      | Tlat          |
           |               |
           |               |
           |               | Ttotal
       -   -  –            |
       |      |            |
       |      |            |
       | Tfp  | Tl1_thru   |
       |      |            |
       |      -            |
       –                   -
       
 - INT execution is difficult for the generator to generate, and difficult to predict, because various rates/latencies and usually some overlap with LD phase. Necessary for precise L1 bound predictions though. Small impact for memory bound cases, so usually omitted
 
 - LD execution phase: time taken to just enqueue the loads. Executed at a constant 1/4cyc throughput, regardless of cache hit/miss. Similar to INT execution: sometimes coissued with FP, overlaps with memory latency. Necessary for precise L1 bound predictions, where latency is short. Can be omitted otherwise

 - Tlat = Tlat_mem + Tlat_L2 = (271 + 200) cyc

 - FP (DP) execution phase: 1/4 throughput, 8cyc latency. 
 - L1 thru: Four 32B L1 cache lines per cycle

 - Assumption: the very first load instruction is a cache miss and loads from memory. All other cache misses hide behind that latency and are considered L1 cache hits. Tlatency therefore overlaps with Tld.
 
 - The FP phase can start as soon as the first load has returned, after Tlat. The L1 cache can now also start the outstanding load delievery phase Tl1_thru, depending on L1 bandwidth. Overlaps with DP phase (not quite perfectly sometimes)


 - Single warp execution time: Ttotal = Tint + max(Tld, Tlat) + max(Tfp, Tl1_thru) 
 

## Multiwarp execution time


- GPUs do not execute single warps (duh)
- Performance: "P = Nflop * Nwarps * clock / Ttotal" is too simple, does not scale with increased warp count
- warps compete for ressources:
  - L2 and memory bandwidth is shared by all 80 SMs.
  - L1 bandwidth is shared by the four quadrants of a SM
  - execution units are are shared by the (up to 16) warps per quadrant
  - In CPU terms: SMT16

- Phase execution time increases, e.g.
  - Tdp = Ndp * 8 * min(1, Nquad / 2)
  - Tl1_thru = Nsm * Ncl_L1 / 4

- This model is also too simple! would correspond to perfectly synchronized phase execution: at every point all warps execute the same phase -> minimum overlap of phases

- Most optimistic model: perfect desynchronization -> maximum overlap. In practice probably not quite perfect desynchronization, but more realistic than synchronicity

- Perfect desync overlap model e.g.:
  - Tdp = Ndp * 8 * min(1,Nquad * Tdp / Ttotal / 2)
  - Tl1_thru = Nsm * Tl1_thru / Ttotal / 4

- Equation for Ttotal now contains Ttotal...
 -> solve iteratively



## Example: 2D Box Stencil with varying range
- Try all block sizes, plot the best block size:

In [ ]:


best_values = []
for r in range(1, 7):
    values = []
    kernel = Kernel2DBoxStencil(stencil_range=r, l1only=True)
    for xblock in [16, 32, 64, 128, 256, 512, 1024]:
        for yblock in [1, 2, 4, 8, 16, 32, 64, 128]:
            if xblock*yblock > 1024 or xblock*yblock < 64:
                continue

            stencilbench.A_gpu_aligned = int(stencilbench.A_gpu)
            stencilbench.B_gpu_aligned = int(stencilbench.B_gpu)
            values.append(stencilbench.benchKernel(kernel, 11, (xblock, yblock, 1))[2])
            
    best_values.append(np.max(np.array(values)))

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(16)
fig.set_figheight(7)


plt.axhline(np.max(np.array(best_values)))

plt.bar(np.arange(1, len(best_values)+1), best_values, width=0.7)


plt.show()

## Data Volume

- How to get the number of loaded L1 cache lines?
- The number of L2 cache lines?


In [ ]:
Kernel2DBoxStencil(stencil_range=1).genAddresses()

In [ ]:
storeVolumes = []
loadVolumes = []
for r in range(1, 10):
    kernel = Kernel2DBoxStencil(stencil_range=r, l1only=True)
    
    measureMetric.measureBandwidthStart()
    block = (32, 8, 1)
    stencilbench.runKernel(kernel, kernel.getGrid(1, block, 15000, 15000), block)
    result = measureMetric.measureMetricStop()
    
    storeVolumes.append(result[1])
    loadVolumes.append(result[0])
    print("mem load: " + str(result[0] / 15000**2))
    print("mem store: " + str(result[1] / 15000**2))
    print("L2  load: " + str(result[2]*32 / 15000**2))
    print("L2  store: " + str(result[3]*32 / 15000**2))
    L2CLs, L1CLs = computeCacheVolumes(kernel, 32, block, (1,1,1))
    
    print("pred L1 CLs: " + str(L1CLs))
    print("pred L2 volume: " + str(L2CLs * 32 / block[0] / block[1] / block[2]))
    print()

In [ ]:
best_values = []

measuredValues = []
predictedValues = []

for r in range(1, 5):
    kernel = Kernel2DBoxStencil(stencil_range=r, l1only=True)
    for xblock in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]:
        for yblock in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]:
            if xblock*yblock > 1024 or xblock*yblock < 32:
                continue

            block = (xblock, yblock, 1)
                        
            print(r, end=" ")
            print(block, end=" ")
            print(block[0] * block[1] * block[2], end="\n")
            stencilbench.A_gpu_aligned = int(stencilbench.A_gpu)
            stencilbench.B_gpu_aligned = int(stencilbench.B_gpu)
            measuredValues.append( stencilbench.benchKernel(kernel, 11, (xblock, yblock, 1))[2])
            predictedValues.append(predictPerformance(kernel, block, (1,1,1), 32) )

            print("{:.0f}  {:.0f}".format( measuredValues[-1], predictedValues[-1]))
            print()
    print()

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(16)
fig.set_figheight(7)


ax.plot(measuredValues, "-x", label="measured")
ax.plot(predictedValues, "-+", label="predicted")
#ax.set_yscale("log")
ax.legend()

plt.show()


In [ ]:
def busyFactor(p):
    return max(1, p)
    #return 1 + (log( 1 + exp((p-1)  )))

def predictPerformance(kernel, block, grid, registers):

    threadsPerBlock = block[0]*block[1]*block[2]
    blocksPerSM = min( 32, int(2**16 / (threadsPerBlock * max(registers, 32)) ))
    warpsPerSM = blocksPerSM * ((threadsPerBlock-1) // 32 + 1)
    warpsPerBlock = ceil(threadsPerBlock / 32)
    
    SMcount = 80 
    clock = 1.38    
    cl_size = 32
    blockShedRate = 0.46
    L2CLs, L1CLs = computeCacheVolumes(kernel, cl_size, block, grid)

    L2CLs += threadsPerBlock / 4
    L1LoadCycles = sum([ len(ld) for ld in L1CLs ]) / 4   / warpsPerBlock
    print(L1LoadCycles)

    # Iitial values
    #Tint = 40 * 5 * max(1, warpsPerSM / 12)
    Tint = 0
    TL1thru = (L1LoadCycles * blocksPerSM) 
    TDP  = kernel.flops * max(1, (warpsPerSM / 8))  * 8
    Tlat_mem = (max(250,  (warpsPerSM*32*16 * SMcount ) / 780 * clock ) if kernel.bytes > 0 else 0)
    Tlat_L2 = (max(200,  (L2CLs * blocksPerSM * SMcount * cl_size ) / 2000 * clock ) if kernel.bytes > 0 else 0)
    Tblocksched = SMcount / 0.5  * blocksPerSM  
    Ttotal = Tblocksched + Tint + max(TDP, TL1thru) + Tlat_mem + Tlat_L2
    print("Tblocksched Tint TL1thru TDP Tlat_mem, Tlat_L2, Ttotal")
    print("{:5.0f} {:5.0f} {:5.0f} {:5.0f} {:5.0f} {:5.0f} {:5.0f}".format(Tblocksched, Tint, TL1thru, TDP, Tlat_mem, Tlat_L2, Ttotal ))
    
    delta = 100
    for i in range(0, 200):
        Tint = 0 #40 * 5 * busyFactor(Tint / Ttotal * warpsPerSM / 12)
        TL1thru = L1LoadCycles  * busyFactor(TL1thru / Ttotal * warpsPerSM)
        TDP  = kernel.flops * busyFactor(warpsPerSM * (TDP / Ttotal) / 8)  * 8 
        Tlat_mem = max(271, Tlat_mem / Ttotal  * (warpsPerSM*32*16 * SMcount ) / 780 * clock) if kernel.bytes > 0 else 0 
        Tlat_L2 = (max(200,  Tlat_L2 / Ttotal  * (L2CLs * blocksPerSM * SMcount * cl_size ) / 2000 * clock ) if kernel.bytes > 0 else 0)
        new_Ttotal = Tblocksched + Tint + max(TDP,  TL1thru) + Tlat_mem + Tlat_L2
        delta = abs(new_Ttotal - Ttotal)
        Ttotal = new_Ttotal
        
        if i > 100 and delta < 0.01:
            break
    
    print("{:5.0f} {:5.0f} {:5.0f} {:5.0f} {:5.0f} {:5.0f} {:5.0f}".format(Tblocksched, Tint, TL1thru, TDP, Tlat_mem, Tlat_L2, Ttotal ))
    return kernel.flops * blocksPerSM * threadsPerBlock * (clock * SMcount / Ttotal )


$$T_{L1thru} = C_{L1loads}  F_{busy}( \frac{T_{L1thru}N_{warps} }{ T_{total}})$$
$$T_{DP} = 8I_{Flops}  F_{busy}( \frac{T_{DP}N_{warps} }{ 8\quad T_{total}})$$
$$T_{lat}^{mem} = \max(271,  \frac{T_{lat}^{mem}}{T_{total}} DV_{mem} / 780 * clock) $$
$$T_{lat}^{L2} = \max(200,  \frac{T_{lat}^{L2}}{T_{total}} * DV_{L2} / 2000 * clock) $$

$$T_{total} = T{blocksched} + T_{int} + max(T_{DP}, T_{L1thru}) + T_{lat}^{mem} + T_{lat}^{L2}$$


In [ ]:
best_values = []

measuredValues = []
predictedValues = []

for r in range(1, 5):
    kernel = Kernel2DBoxStencil(stencil_range=r)
    
    for xblock in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]:
        for yblock in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]:
            if xblock*yblock > 1024 or xblock*yblock < 32:
                continue

            block = (xblock, yblock, 1)
                        
            print(r, end=" ")
            print(block, end=" ")
            print(block[0] * block[1] * block[2], end="\n")
            stencilbench.A_gpu_aligned = int(stencilbench.A_gpu)
            stencilbench.B_gpu_aligned = int(stencilbench.B_gpu)
            measuredValues.append( stencilbench.benchKernel(kernel, 11, (xblock, yblock, 1))[2])
            predictedValues.append(predictPerformance(kernel, block, (1,1,1), 32) )

            print("{:.0f}  {:.0f}".format( measuredValues[-1], predictedValues[-1]))
            print()
    print()

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(16)
fig.set_figheight(7)


ax.plot(measuredValues, "-x", label="measured")
ax.plot(predictedValues, "-+", label="predicted")
#ax.set_yscale("log")
ax.legend()

plt.show()


In [ ]:
print(Kernel2DBoxStencil(stencil_range=r).text)